In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from scipy.sparse import vstack, hstack
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

In [3]:
dataset = pd.read_csv('Output/data_clean&processed_addedsentiment.csv')
# dataset = dataset.drop('Unnamed: 0', axis = 1)

dataset.columns

Index(['id_row', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type', 'story', 'faq', 'num_faq', 'comments', 'n_comments',
       'duration', 'parent_category', 'category_name', 'location_name',
       'month_launched', 'year_launched', 'backers_count_log', 'goal_log',
       'pledged_log', 'duration_log', 'n_comments_log', 'num_faq_bool',
       'avg_fund_per_backer', 'comments_afinn', 'blurb_name', 'faq_comments',
       'story_afinn', 'faq_comments_afinn', 'blurb_name_afinn', 'blurb_afinn',
       'name_afinn'],
      dtype='object')

In [4]:
dataset.head()

,id_row,backers_count,blurb,category,converted_pledged_amount,country,created_at,current_currency,deadline,fx_rate,...,num_faq_bool,avg_fund_per_backer,comments_afinn,blurb_name,faq_comments,story_afinn,faq_comments_afinn,blurb_name_afinn,blurb_afinn,name_afinn
0,0,1,With your help we will create this device that...,"{'id': 331, 'name': '3D Printing', 'slug': 'te...",1,ES,2015-08-18 21:01:19,USD,2016-07-09 20:11:23,1.212886,...,0,1.0,8.0,With your help we will create this device that...,[' ']AnaTalifi Wall Nazur\nabout 2 years ago...,-10.0,8.0,4.0,2.0,2.0
1,1,2,We at Ormiston Primary are looking at starting...,"{'id': 309, 'name': 'Farms', 'slug': 'food/far...",9,NZ,2015-08-11 18:04:21,USD,2015-09-11 15:55:41,0.723585,...,0,7.5,-1.0,We at Ormiston Primary are looking at starting...,[' ']No comments yet.,-10.0,-1.0,1.0,1.0,0.0
2,2,0,Self-taught aspiring metalsmith Looking for he...,"{'id': 54, 'name': 'Mixed Media', 'slug': 'art...",0,US,2015-04-28 21:14:03,USD,2015-05-28 21:14:00,1.000000,...,0,0.0,7.0,Self-taught aspiring metalsmith Looking for he...,[' ']Bambi Lewis\nover 7 years ago\nBeautifu...,-10.0,7.0,4.0,2.0,2.0
3,3,0,So many women believe they are past their prim...,"{'id': 278, 'name': 'People', 'slug': 'photogr...",0,US,2014-07-07 01:30:52,USD,2014-10-26 00:00:48,1.000000,...,0,0.0,139.0,So many women believe they are past their prim...,[' ']paul wheatonCreator\nover 2 years ago\n...,-10.0,139.0,3.0,0.0,3.0
4,4,10,The Horror Zine's Jeani Rector brings us anoth...,"{'id': 324, 'name': 'Anthologies', 'slug': 'pu...",340,US,2014-11-04 16:30:58,USD,2014-12-09 09:20:35,1.000000,...,0,34.0,-1.0,The Horror Zine's Jeani Rector brings us anoth...,[' ']No comments yet.,-10.0,-1.0,4.0,4.0,0.0


In [5]:
# numerical_features = ['goal','converted_pledged_amount','backers_count','backers_count_log','goal_log','pledged_log','duration_log','n_comments_log','avg_fund_per_backer','comments_afinn','num_faq_bool','story_afinn','faq_comments_afinn', 'blurb_name_afinn', 'blurb_afinn','name_afinn']

In [6]:
numerical_features = ['goal','converted_pledged_amount','backers_count','backers_count_log','goal_log','pledged_log','duration_log', 'avg_fund_per_backer', 'blurb_afinn','name_afinn']

In [19]:
X = dataset[numerical_features]
y = dataset.state
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


numerical_logreg = LogisticRegression()
numerical_logreg.fit(X_train, y_train)
y_pred_class = numerical_logreg.predict(X_test)
y_pred_class.sum()
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
print('Confusion Matrix: \n', metrics.confusion_matrix(y_test, y_pred_class))
pd.DataFrame(
        [x for x in zip(list(X_train.columns) + [str(x) for x in range(len(X_train.columns))], numerical_logreg.coef_[0])], 
        columns=['features', 'coefficients']
    ).sort_values(
        by = 'coefficients', 
        ascending = False, 
        key = lambda x: abs(x)
        )

Accuracy:  0.9157894736842105
Confusion Matrix: 
 [[281  31]
 [ 41 502]]


,features,coefficients
5,pledged_log,0.752619
3,backers_count_log,0.721604
4,goal_log,-0.515935
6,duration_log,-0.302962
8,blurb_afinn,-0.136756
9,name_afinn,-0.086673
2,backers_count,0.012102
7,avg_fund_per_backer,0.000727
1,converted_pledged_amount,0.000367
0,goal,-0.000009


In [28]:
X = dataset[numerical_features]
y = dataset.state
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

c_values = [100, 10, 1.0, 0.5, 0.1, 0.01]

numerical_logreg = LogisticRegression(random_state=42, penalty='l2', dual=False, tol=0.0001, C=0.01, fit_intercept=True, intercept_scaling=1.0, class_weight=None)
numerical_logreg.fit(X_train, y_train)
y_pred_class = numerical_logreg.predict(X_test)
y_pred_class.sum()
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
print('Confusion Matrix: \n', metrics.confusion_matrix(y_test, y_pred_class))
pd.DataFrame(
        [x for x in zip(list(X_train.columns) + [str(x) for x in range(len(X_train.columns))], numerical_logreg.coef_[0])], 
        columns=['features', 'coefficients']
    ).sort_values(
        by = 'coefficients', 
        ascending = False, 
        key = lambda x: abs(x)
        )

Accuracy:  0.9368421052631579
Confusion Matrix: 
 [[275  37]
 [ 17 526]]


,features,coefficients
5,pledged_log,0.796668
3,backers_count_log,0.763932
4,goal_log,-0.591608
6,duration_log,-0.330946
9,name_afinn,-0.063041
2,backers_count,0.032329
8,blurb_afinn,0.018799
7,avg_fund_per_backer,0.002637
1,converted_pledged_amount,-0.000023
0,goal,-0.000014


In [29]:
import pickle

f = open('logreg.pckl', 'wb')
pickle.dump(numerical_logreg, f)
f.close()